# Transformação de Dados Meteorológicos NOAA ISD

## Configuração da Sessão Spark\n
Inicializa a sessão Spark. A configuração exata pode precisar de ajustes dependendo do ambiente (Databricks, Synapse).

In [ ]:
from pyspark.sql import SparkSession\n
\n
# Cria ou obtém uma sessão Spark existente\n
spark = SparkSession.builder\\\n
    .appName(\
)\\\n
    .getOrCreate()\n
\n
print(f\
)

## Leitura dos Dados Ingeridos\n
Lê os arquivos Parquet do NOAA ISD que foram copiados pelo pipeline de ingestão para o contêiner 'output'.

In [ ]:
# Substitua <STORAGE_ACCOUNT_NAME> pelo nome da sua conta de armazenamento (datavalidation456)\n
storage_account_name = \
\n
container_name = \
\n
input_path = f\